In [1]:
  #!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as nc
from gi.repository import NumCosmoMath as ncm
import numpy as np
from astropy.io import fits
from astropy.table import Table
import sys
sys.path.insert(0,'../../scripts')

import pyccl as ccl

from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec

ncm.cfg_init()
ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

initialize the `Cosmological models`

In [2]:
#CCL cosmology
cosmo_ccl = ccl.Cosmology(Omega_c = 0.30711 - 0.048254, Omega_b = 0.048254, h = 0.677, sigma8 = 0.8822714165197718, n_s=0.96, Omega_k = 0, transfer_function='eisenstein_hu')
#ccl_cosmo_set_high_prec (cosmo_ccl)

cosmo_numcosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj (cosmo_ccl)

psf = hmfunc.peek_psf ()

# Define proxy modelling
Use a mass proxy, define the probability for observing a proxy given a mass and redhsift

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
\mu(M,z) = \mu_0 + a_\mu^M\log_{10}\frac{M}{M_0} + a_\mu^z\log_{10}\frac{1+z}{1+z_0} 
$$
variance is
$$
\sigma(M,z) = \sigma_0 + a_\sigma^M\log_{10}\frac{M}{M_0} + a_\sigma ^z\log_{10}\frac{1+z}{1+z_0} 
$$

In [3]:
#CosmoSim_proxy model
#M_0, z_0
theta_pivot = [3e14/0.71, 0.6]
#\mu_0, a_\mu^z, a_\mu^M
theta_mu = [3.19, -0.7, 2]
#\sigma_0, a_\sigma^z, a_\sigma^M
theta_sigma = [0.33, 0.,-0.08]
#Richness object

area = (0.25)*4*np.pi / 100.0
lnRl = 1.0
lnRu = 2.0
zl = 0.25
zu = 1.0

#Numcosmo_proxy model
cluster_z = nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (zl, zu))
cluster_m = nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, lnRl, lnRu))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))

initialize the `ClusterAbundance` object

In [4]:
#Numcosmo Cluster Abundance

#First we need to define the multiplicity function here we will use the tinker
mulf = nc.MultiplicityFuncTinker.new()
mulf.set_linear_interp (True)
mulf.set_mdef(nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
#Second we need to construct a filtered power spectrum 

hmf = nc.HaloMassFunction.new(dist,psf,mulf)
hmf.set_area(area)

ca = nc.ClusterAbundance.new(hmf,None)
mset = ncm.MSet.new_array([cosmo_numcosmo,cluster_m,cluster_z])

In [5]:
ncount = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
ncount.catalog_load ("ncount_ascaso.fits")

In [6]:
cosmo_numcosmo.props.Omegac_fit = True
cosmo_numcosmo.props.w0_fit = True
cluster_m.props.mup0_fit = True
mset.prepare_fparam_map ()

In [7]:
ncount.set_binned (False)

In [8]:
dset = ncm.Dataset.new ()
dset.append_data (ncount)

In [9]:
lh = Ncm.Likelihood (dataset = dset)
fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD)

In [10]:
Ncm.func_eval_set_max_threads (3)
print (Ncm.func_eval_log_pool_stats ())


init_sampler = Ncm.MSetTransKernGauss.new (0)
init_sampler.set_mset (mset)
init_sampler.set_prior_from_mset ()
init_sampler.set_cov_from_rescale (1.0)

nwalkers = 300

walker = Ncm.FitESMCMCWalkerAPES.new (nwalkers, mset.fparams_len ())
walker.set_over_smooth (0.37)

esmcmc = Ncm.FitESMCMC.new (fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE)
esmcmc.set_nthreads (3)

esmcmc.set_data_file ("ncount_ascaso_mcmc_unbinned_calib.fits")

esmcmc.start_run ()
esmcmc.run_lre (50, 1.0e-3)
esmcmc.end_run ()

esmcmc.mean_covar ()
fit.log_covar ()


# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      3
None
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensamble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0300.
#   Number of threads: 0003.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Skipping 17400 points (58.000000 iterations), will start at 17401-th point.
#----------------------------------------------------------------------------------
# NcmFitESMCMC: validating catalog rows [17100, 17400)
# NcmFitESMCMC: samples rows [17100, 17400) successfully validated.
# NcmFitESMCMC: Largest relative error 1.000000e-03 not attained: 2.108963e-03
# NcmFitESMCMC: ln (eVol) =                    nan; ln (gVol) =       -6.7230334116489; lnNorm =   

# NcmFitESMCMC: Largest relative error 1.000000e-03 not attained: 1.914228e-03
# NcmFitESMCMC: ln (eVol) =                    nan; ln (gVol) =      -6.76823150560413; lnNorm =                    nan
# NcmFitESMCMC: Running more 19 runs...
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Calculating [000019] Ensemble Sampler Markov Chain Monte Carlo runs [APES-Move:VKDE:Cauchy]
# Task:NcmFitESMCMC, started at: Mon Apr 18 2022, 11:25:56
# NcmMSetCatalog: Current mean:  -3062.2       0.27402     -0.95379      3.1612     
# NcmMSetCatalog: Current msd:    0.63825      0.00045202   0.0018126    0.0022995  
# NcmMSetCatalog: Current sd:     74.109       0.026188     0.097309     0.11062    
# NcmMSetCatalog: Current var:    5492.2       0.00068581   0.0094691    0.012237   
# NcmMSetCatalog: Current tau:    1.5354       6.1671       7.1827       8.9445     
# NcmMSetCatalog: Current skfac:  0.99865      1.0242       1.0171       1.0249     


# NcmFitESMCMC:acceptance ratio 42.8596% (last update 43.0000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble variance of -2ln(L):       5.46216525392806 (2n = 6), min(-2ln(L)) =      -3071.81741060536.
# Task:NcmFitESMCMC, completed: 5700 of 8700, elapsed time: 00:12:20.8830
# Task:NcmFitESMCMC, mean time: 00:00:00.1300 +/- 00:00:00.0298
# Task:NcmFitESMCMC, time left: 00:06:29.9167 +/- 00:01:29.3416
# Task:NcmFitESMCMC, current time:        Mon Apr 18 2022, 11:38:17
# Task:NcmFitESMCMC, estimated to end at: Mon Apr 18 2022, 11:44:47 +/- 00:01:29.3416
# NcmMSetCatalog: Current mean:  -3063         0.27394     -0.95376      3.1617     
# NcmMSetCatalog: Current msd:    0.5649       0.00042709   0.0017123    0.0021711  
# NcmMSetCatalog: Current sd:     69.739       0.026308     0.097733     0.11105    
# NcmMSetCatalog: Current var:    4863.6       0.0006921    0.0095517    0.012332   
# NcmMSetCatalog: Current tau:    1.5354       6.1671       7.1827   

# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 42.2024% (last update 38.3333%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble variance of -2ln(L):       5.60679364425126 (2n = 6), min(-2ln(L)) =      -3071.81741060536.
# Task:NcmFitESMCMC, completed: 8400 of 8700, elapsed time: 00:18:16.9954
# Task:NcmFitESMCMC, mean time: 00:00:00.1306 +/- 00:00:00.0246
# Task:NcmFitESMCMC, time left: 00:00:39.1773 +/- 00:00:07.3938
# Task:NcmFitESMCMC, current time:        Mon Apr 18 2022, 11:44:13
# Task:NcmFitESMCMC, estimated to end at: Mon Apr 18 2022, 11:44:52 +/- 00:00:07.3938
# NcmMSetCatalog: Current mean:  -3063.6       0.27386     -0.9539       3.1622     
# NcmMSetCatalog: Current msd:    0.50668      0.00040635   0.0016221    0.0020619  
# NcmMSetCatalog: Current sd:     66.062       0.026435     0.097782     0.11138    
# NcmMSetCatalog: Current var:    4364.1       0.0006988    0.0095613    0.012405   
# NcmMSetCatalog: Curr

# NcmMSetCatalog: Current msd:    0.46346      0.00038672   0.0014842    0.0017601  
# NcmMSetCatalog: Current sd:     63.239       0.026467     0.097532     0.11099    
# NcmMSetCatalog: Current var:    3999.2       0.00070052   0.0095126    0.012318   
# NcmMSetCatalog: Current tau:    1.5307       6.0843       6.5995       7.1675     
# NcmMSetCatalog: Current skfac:  0.99904      1.0182       1.0128       1.0153     
# NcmMSetCatalog: Maximal Shrink factor = 1.01855               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `95':
# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 42.3063% (last update 43.6667%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble variance of -2ln(L):       5.52219389926676 (2n = 6), min(-2ln(L)) =      -3071.81741060536.
# Task:NcmFitESMCMC, completed: 11100 of 13200, elapse

...

# NcmMSetCatalog: Current var:    2609.5       0.0007083    0.0096465    0.012148   
# NcmMSetCatalog: Current tau:    1.5248       5.8508       5.8546       6.793      
# NcmMSetCatalog: Current skfac:  0.99938      1.0109       1.0093       1.0106     
# NcmMSetCatalog: Maximal Shrink factor = 1.01145               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `146':
# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 42.2500% (last update 46.6667%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble variance of -2ln(L):       5.62297621744024 (2n = 6), min(-2ln(L)) =      -3071.81741060536.
# Task:NcmFitESMCMC, completed: 26400 of 26400, elapsed time: 00:55:25.0235
# Task:NcmFitESMCMC, mean time: 00:00:00.1259 +/- 00:00:00.0134
# Task:NcmFitESMCMC, time left: 00:00:00.0000 +/- 00:00:00.0000
# Task:NcmFitESMCMC

# Task:NcmFitESMCMC, completed: 29400 of 29400, elapsed time: 01:00:52.8871
# Task:NcmFitESMCMC, mean time: 00:00:00.1242 +/- 00:00:00.0126
# Task:NcmFitESMCMC, time left: 00:00:00.0000 +/- 00:00:00.0000
# Task:NcmFitESMCMC, current time:        Mon Apr 18 2022, 12:26:49
# Task:NcmFitESMCMC, estimated to end at: Mon Apr 18 2022, 12:26:49 +/- 00:00:00.0000
# NcmFitESMCMC: Largest relative error 1.000000e-03 not attained: 1.136806e-03
# NcmFitESMCMC: ln (eVol) =                    nan; ln (gVol) =      -6.93829591721107; lnNorm =                    nan
# NcmFitESMCMC: Running more 10 runs...
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Calculating [000010] Ensemble Sampler Markov Chain Monte Carlo runs [APES-Move:VKDE:Cauchy]
# Task:NcmFitESMCMC, started at: Mon Apr 18 2022, 11:25:56
# NcmMSetCatalog: Current mean:  -3065.9       0.27444     -0.95276      3.16       
# NcmMSetCatalog: Current msd:    0.27847      0.0002916    0.00107

# Task:NcmFitESMCMC, started at: Mon Apr 18 2022, 11:25:56
# NcmMSetCatalog: Current mean:  -3066.1       0.27437     -0.95296      3.1601     
# NcmMSetCatalog: Current msd:    0.2619       0.00028393   0.0010244    0.0012423  
# NcmMSetCatalog: Current sd:     47.641       0.0266       0.098201     0.10981    
# NcmMSetCatalog: Current var:    2269.7       0.00070756   0.0096435    0.012057   
# NcmMSetCatalog: Current tau:    1.5232       5.7424       5.4842       6.4507     
# NcmMSetCatalog: Current skfac:  0.99948      1.0095       1.0084       1.0094     
# NcmMSetCatalog: Maximal Shrink factor = 1.00983               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `168':
# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 42.1758% (last update 45.3333%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble va

# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 42.1778% (last update 37.6667%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble variance of -2ln(L):        4.9307034130775 (2n = 6), min(-2ln(L)) =      -3071.81741060536.
# Task:NcmFitESMCMC, completed: 36000 of 38400, elapsed time: 01:12:20.8720
# Task:NcmFitESMCMC, mean time: 00:00:00.1206 +/- 00:00:00.0110
# Task:NcmFitESMCMC, time left: 00:04:49.3897 +/- 00:00:26.4920
# Task:NcmFitESMCMC, current time:        Mon Apr 18 2022, 12:38:17
# Task:NcmFitESMCMC, estimated to end at: Mon Apr 18 2022, 12:43:07 +/- 00:00:26.4920
# NcmMSetCatalog: Current mean:  -3066.3       0.2744      -0.95287      3.16       
# NcmMSetCatalog: Current msd:    0.24445      0.00027098   0.00098455   0.0011632  
# NcmMSetCatalog: Current sd:     46.035       0.026563     0.098355     0.10938    
# NcmMSetCatalog: Current var:    2119.2       0.0007056    0.0096738    0.011964   
# NcmMSetCatalog: Cu

# NcmMSetCatalog: Current msd:    0.23159      0.00026354   0.00095216   0.0011289  
# NcmMSetCatalog: Current sd:     44.814       0.026513     0.098461     0.10924    
# NcmMSetCatalog: Current var:    2008.3       0.00070295   0.0096946    0.011934   
# NcmMSetCatalog: Current tau:    1.5223       5.6318       5.3304       6.087      
# NcmMSetCatalog: Current skfac:  0.99953      1.0081       1.0071       1.0087     
# NcmMSetCatalog: Maximal Shrink factor = 1.00902               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `190':
# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 42.1742% (last update 40.0000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble variance of -2ln(L):       5.52919863958464 (2n = 6), min(-2ln(L)) =      -3071.81741060536.
# Task:NcmFitESMCMC, completed: 39600 of 41400, elaps

In [11]:
ntests   = 100.0
nwalkers = 300
burnin   = 80
mcat = Ncm.MSetCatalog.new_from_file_ro ("ncount_ascaso_mcmc_unbinned.fits", nwalkers * burnin)

mcat.log_current_chain_stats ()
mcat.calc_max_ess_time (ntests, Ncm.FitRunMsgs.FULL);
mcat.calc_heidel_diag (ntests, 0.0, Ncm.FitRunMsgs.FULL);

mset.pretty_log ()
mcat.log_full_covar ()
mcat.log_current_stats ()

be, post_lnnorm_sd = mcat.get_post_lnnorm ()
lnevol, glnvol = mcat.get_post_lnvol (0.6827)

Ncm.cfg_msg_sepa ()
print ("# Bayesian evidence:                                 % 22.15g +/- % 22.15g" % (be, post_lnnorm_sd))
print ("# 1 sigma posterior volume:                          % 22.15g" % lnevol)
print ("# 1 sigma posterior volume (Gaussian approximation): % 22.15g" % glnvol)

# NcmMSetCatalog: Current skfac:  1.0142       1.0165       1.0177       1.0151     
# NcmMSetCatalog: Maximal Shrink factor = 1.01897               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `236':
# NcmMSetCatalog: Constant break point at `0':
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Calculating catalog effective sample size from chain 236 => 0 using 100 blocks:
# NcmMSetCatalog: - best cutoff time:         0   
# NcmMSetCatalog: - total number of points:   236  (70800)
# NcmMSetCatalog: - number of points left:    236  (70800)
# NcmMSetCatalog: - worst parameter:          NcmFit:m2lnL[00]
# NcmMSetCatalog: - worst parameter ess:      34.47
# NcmMSetCatalog: - worst parameter ar order: 1   
# NcmMSetCatalog: - ess's:                    34.47 38.49 43.95 35.81
#----------------------------------------